In [1]:
import os
import sys
from sambanova_endpoint import SambaNovaEndpoint, SambaverseEndpoint, SambaStudioFastCoE, SambaNovaEmbeddingModel
from dotenv import load_dotenv
import json
load_dotenv("../.env")


True

# SambaStudio endpoint

## Non streaming

In [2]:

llm = SambaNovaEndpoint(
        streaming=False,
        #base_uri="api/predict/generic",
        model_kwargs={
            "do_sample": True, 
            "temperature": 0.01,
            "max_tokens_to_generate": 256,
            "process_prompt": False,
            "select_expert": "llama-2-7b-chat-hf",
            # "repetition_penalty": 1.0,
            # "top_k": 50",
            # "top_logprobs": 0,
            # "top_p": 1.0
        }
    ) 


In [3]:
llm.invoke("tell me a 50 word tale")

'of a character who is a master of disguise\n\nSure! Here is a 50-word tale of a character who is a master of disguise:\n\n"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate\'s lair, her shimmering scales and flowing locks fooling even the most skeptical of eyes."'

## Streaming

In [4]:
llm=SambaNovaEndpoint(
        streaming=True,
        model_kwargs={
            "do_sample": True,
            "max_tokens_to_generate": 256,
            "temperature": 0.01,
            "process_prompt": False,
            "select_expert": "llama-2-7b-chat-hf",
            # "repetition_penalty": 1.0,
            # "top_k": 50",
            # "top_logprobs": 0,
            # "top_p": 1.0
            }
        )

In [5]:
for chunk in llm.stream("tell me a 50 word tale"):
    print(chunk, end="", flush=True)

 of a character who is a master of disguise

Sure! Here is a 50-word tale of a character who is a master of disguise:

"Araxys, the skilled disguise artist, transformed into a stunning mermaid to infiltrate a pirate's lair, then morphed into a loyal servant to gain the trust of the pirate's crew."

# Sambaverse


## Non Streaming

In [6]:
llm = SambaverseEndpoint(
    sambaverse_model_name = "Meta/llama-2-7b-chat-hf",
    model_kwargs={
        "do_sample": True, 
        "max_tokens_to_generate": 256,
        "temperature": 0.01,
        "process_prompt": True,
        "select_expert": "llama-2-7b-chat-hf",
        #"stop_sequences": '\"sequence1\",\"sequence2\"',
        #"repetition_penalty":  1.0,
        #"top_k": 50,
        #"top_p": 1.0
        }
    ) 

In [7]:
llm.invoke("tell me a 50 word tale")

' Sure! Here is a 50-word tale:\n\nThe cat purred contentedly on my lap, pawing at my hand with a gentle mew.'

## Streaming

In [8]:
llm = SambaverseEndpoint(
    streaming=True,
    sambaverse_model_name="Meta/llama-2-7b-chat-hf",
    model_kwargs={
        "do_sample": True, 
        "max_tokens_to_generate": 256,
        "temperature": 0.01,
        "process_prompt": True,
        "select_expert": "llama-2-7b-chat-hf",
        #"stop_sequences": '\"sequence1\",\"sequence2\"',
        #"repetition_penalty":  1.0,
        #"top_k": 50,
        #"top_p": 1.0
        }
    ) 

In [9]:

for chunk in llm.stream("tell me a 50 word tale"):
    print(chunk, end="", flush=True)

  Sure! Here is a 50-word tale:

The cat purred contentedly on my lap, pawing at my hand with a gentle mew.

# FastCoe endpoint

In [2]:
llm = SambaStudioFastCoE(model='llama3-8b')

In [3]:
import json
llm.invoke(json.dumps([{'role': 'user', 'content': 'hello'}]))

"Hello! It's great to meet you! I'm here to assist you with any questions or topics you'd like to discuss. I'm a helpful assistant developed by SambaNova Systems, and I'm committed to providing you with accurate and respectful information. What's on your mind today?"

In [5]:
llm.invoke('hello')

"Hello! It's great to meet you! I'm here to assist you with any questions or topics you'd like to discuss. I'm a helpful assistant developed by SambaNova Systems, and I'm committed to providing you with accurate and respectful information. What's on your mind today?"

In [4]:
for i in llm.stream('hello tell me a long story'):
    print(i)


I'd be delighted to spin a yarn for you! Here's a tale that'll take us on 
a journey across the globe, through time, and into the hearts of those who lived through 
it.

Once upon a time, in the early 19th century, there was a small village nestled in 
the rolling hills of rural France. The villagers lived simple lives, working the land, tending to 
their animals, and trading goods with neighboring towns. Among them was a young girl named Sophie, with 
hair as golden as the sun and eyes as blue as the sky.

Sophie was a curious and 
adventurous soul, always eager to explore the world beyond her village. She spent her days helping her mother 
with the household chores, learning the secrets of traditional French cooking, and listening to the stories of the 
village elder, Monsieur LeFleur.

One day, a group of travelers passed through the 
village, seeking refuge from the turmoil of the Napoleonic Wars. Among them was a young man named 
Alexandre, a skilled musician with a voice as smoo

# Embeddings

In [ ]:
embedding = SambaNovaEmbeddingModel()
embedding.embed_documents(["tell me a 50 word tale","tell me a joke"])
embedding.embed_query("tell me a 50 word tale")

In [13]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

docs=["tell me a 50 word tale","tell me a joke","when was America discoverd?", "how to build an engine?", "give me 3 party activities", "give me three healty dishes"]
docs=[Document(doc) for doc in docs]

query = "prompt for generating something fun"

vectordb = Chroma.from_documents(docs, embedding)
retriever = vectordb.as_retriever()

retriever.get_relevant_documents(query)


<bound method Response.json of <Response [200]>>


[Document(page_content='give me 3 party activities'),
 Document(page_content='tell me a 50 word tale'),
 Document(page_content='tell me a joke'),
 Document(page_content='give me three healty dishes')]

In [5]:
embedding = SambaNovaEmbeddingModel(
    batch_size=1,
    model_kwargs = {
        "select_expert":"e5-mistral-7b-instruct"
        }
    )

In [ ]:
embedding.embed_documents(["tell me a 50 word tale","tell me a joke"])
embedding.embed_query("tell me a 50 word tale")